In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO
from sys import getsizeof

Load the image and display it

In [ ]:
# image is originally from https://unsplash.com/photos/XgcdAE1Gqlg
url = 'https://www.cs.columbia.edu/~djhsu/coms3251-f22/data/chenyu-guan-XgcdAE1Gqlg-unsplash.png'
data = BytesIO(urlopen(url).read())
img = plt.imread(data)
plt.imshow(img)
plt.show()

The memory size of `img` (in bytes) can be obtained using `getsizeof`

In [ ]:
getsizeof(img)

The shape of `img` shows that it can be thought of as a stack of matrices, one per color channel (red, green, blue).

In [ ]:
img.shape

In [ ]:
img[:,:,0].shape # red channel

In [ ]:
img[:,:,1].shape # green channel

In [ ]:
img[:,:,2].shape # blue channel

Compute the singular values of the red color channel, and plot them on a logarithmic scale against the index.

In [ ]:
U,s,VT = np.linalg.svd(img[:,:,0], full_matrices=False)
ax = plt.gca()
ax.plot(range(1,1+len(s)), np.log(1+s))
ax.set_xlabel('$i$')
ax.set_ylabel('$\log(1+\sigma_i)$')
ax.set_title('Singular values of red color channel')
plt.show()

Implement a function that takes as input an image and returns a "compressed" image in which each color channel is implicitly represented by a rank $k$ matrix (where $k$ is specified by the user).

In [ ]:
def compressed_image(k, img):
    """Compute rank k approximations of each channel of a given m-by-n pixel image
    
    The rank k approximation of a color channel is represented the matrix product M @ N, where M has shape (m,k)
    and N has shape (k,n).

    @param k: the rank of the approximation
    @param img: numpy ndarray with shape (m,n,3)
    @return (left,right): tuple of numpy ndarrays, where left has shape (m,k,3), and right has shape (k,n,3)
    """

    left = np.empty((img.shape[0],k,3))
    right = np.empty((k,img.shape[1],3))

    u_red,s_red,vt_red = np.linalg.svd(img[:,:,0], full_matrices=False)
    u_green,s_green,vt_green = np.linalg.svd(img[:,:,1], full_matrices=False)
    u_blue,s_blue,vt_blue = np.linalg.svd(img[:,:,2], full_matrices=False)

    left = np.stack((u_red[:, :k]@np.diag(s_red[:k]), u_green[:, :k]@np.diag(s_green[:k]), u_blue[:, :k]@np.diag(s_blue[:k])), axis=2)
    # right = np.stack((np.diag(s_red[:k])@vt_red[:k, :], np.diag(s_green[:k])@vt_green[:k, :], np.diag(s_blue[:k])@vt_blue[:k, :]), axis=2)

    # left = np.dstack((u_red[:, :k], u_green[:, :k], u_blue[:, :k]))
    right = np.dstack((vt_red[:k, :], vt_green[:k, :], vt_blue[:k, :]))
    
    return (left,right)

We can compute the memory size of the compressed representation as follows.

In [ ]:
left_right_tuple = compressed_image(8, img) # k=8
getsizeof(left_right_tuple[0]) + getsizeof(left_right_tuple[1])

In [ ]:
left_right_tuple = compressed_image(32, img) # k=32
getsizeof(left_right_tuple[0]) + getsizeof(left_right_tuple[1])

Display the compressed versions of the image

In [ ]:
def uncompress(left_right_tuple):
    left = left_right_tuple[0]
    right = left_right_tuple[1]
    img = np.empty((left.shape[0],right.shape[1],3))
    for i in range(3):
        img[:,:,i] = left[:,:,i] @ right[:,:,i]
     # it is possible that some entries of img will be outside the range [0..1], so "clip" the values to the range
    return np.clip(img,0.,1.)

In [ ]:
plt.imshow(uncompress(compressed_image(8, img)))

In [ ]:
plt.imshow(uncompress(compressed_image(16, img)))

In [ ]:
plt.imshow(uncompress(compressed_image(32, img)))

In [ ]:
plt.imshow(uncompress(compressed_image(64, img)))